# Basics

In [4]:
%%writefile program.cpp
#include<iostream>
using namespace std;
int main(){
  cout << "HELLO" << endl;
  return 0;
}

Overwriting program.cpp


In [5]:
!g++ program.cpp -o program.exe

In [3]:
!./program.exe

HELLO


# Task 1: Vector Addition

In [29]:
%%writefile vector_add.cu
#include <cuda.h>
#include <stdio.h>

__global__ void vectorAdd(int *a, int *b, int *c, int n) {
    int i = threadIdx.x + blockIdx.x * blockDim.x;
    if (i < n) {
        c[i] = a[i] + b[i];
    }
}

int main() {
    int n = 10;
    int size = n * sizeof(int);

    int *h_a, *h_b, *h_c; // Host pointers
    int *d_a, *d_b, *d_c; // Device pointers

    // Allocate memory on the host
    h_a = (int *)malloc(size);
    h_b = (int *)malloc(size);
    h_c = (int *)malloc(size);

    // Initialize host arrays
    for (int i = 0; i < n; i++) {
        h_a[i] = i;
        h_b[i] = i;
    }

    // Allocate memory on the device
    cudaMalloc(&d_a, size);
    cudaMalloc(&d_b, size);
    cudaMalloc(&d_c, size);

    // Copy data from host to device
    cudaMemcpy(d_a, h_a, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, h_b, size, cudaMemcpyHostToDevice);

    // Launch kernel
    int threadsPerBlock = 256;
    int blocksPerGrid = (n + threadsPerBlock - 1) / threadsPerBlock;
    vectorAdd<<<blocksPerGrid, threadsPerBlock>>>(d_a, d_b, d_c, n);

    // Copy result back to host
    cudaMemcpy(h_c, d_c, size, cudaMemcpyDeviceToHost);

    // Print vector A
    printf("\nVector A: ");
    for (int i = 0; i < n; i++) {
        printf("%d ", h_a[i]);
    }

    // Print vector B
    printf("\nVector B: ");
    for (int i = 0; i < n; i++) {
        printf("%d ", h_b[i]);
    }

    // Print result
    printf("\nSum Vector: ");
    for (int i = 0; i < n; i++) {
        printf("%d ", h_c[i]);
    }

    // Free memory
    free(h_a); free(h_b); free(h_c);
    cudaFree(d_a); cudaFree(d_b); cudaFree(d_c);

    return 0;
}


Overwriting vector_add.cu


In [30]:
!nvcc vector_add.cu -o vector_add
!./vector_add


Vector A: 0 1 2 3 4 5 6 7 8 9 
Vector B: 0 1 2 3 4 5 6 7 8 9 
Sum Vector: 0 2 4 6 8 10 12 14 16 18 

# Task 2: Matrix Multiplication

In [26]:
%%writefile matrix_mul.cu
#include <cuda.h>
#include <stdio.h>

__global__ void matrixMul(int *a, int *b, int *c, int n) {
    // find row and column number
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;

    // 1 thread will find 1 index of product matrix
    if(row < n && col < n) {
        int sum = 0;
        for(int i = 0; i < n; i++) {
            sum += a[row * n + i] * b[i * n + col];
        } c[row*n + col] = sum;
    }
}

int main() {
    int n = 4;
    int N = n*n;
    int size = N * sizeof(int);

    int *h_a, *h_b, *h_c; // Host pointers
    int *d_a, *d_b, *d_c; // Device pointers

    // Allocate memory on the host
    h_a = (int *)malloc(size);
    h_b = (int *)malloc(size);
    h_c = (int *)malloc(size);

    // Initialize host arrays
    for (int i = 0; i < N; i++) {
        h_a[i] = i;
        h_b[i] = i;
    }

    // Allocate memory on the device
    cudaMalloc(&d_a, size);
    cudaMalloc(&d_b, size);
    cudaMalloc(&d_c, size);

    // Copy data from host to device
    cudaMemcpy(d_a, h_a, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, h_b, size, cudaMemcpyHostToDevice);

    // Launch kernel
    dim3 blockDim(16,16);
    dim3 gridDim((n + blockDim.x - 1) / blockDim.x, (n + blockDim.y - 1) / blockDim.y);
    matrixMul<<<gridDim, blockDim>>>(d_a, d_b, d_c, n);

    // Copy result back to host
    cudaMemcpy(h_c, d_c, size, cudaMemcpyDeviceToHost);


    // Print matrix A
    printf("Matrix A:\n");
    for (int i = 0; i < n; i++) {
        for(int j = 0; j < n; j++){
          printf("%d ", h_a[i * n + j]);
        } printf("\n");
    } printf("\n");

    // Print matrix B
    printf("Matrix B:\n");
    for (int i = 0; i < n; i++) {
        for(int j = 0; j < n; j++){
          printf("%d ", h_b[i * n + j]);
        } printf("\n");
    } printf("\n");


    // Print result
    printf("Product:\n");
    for (int i = 0; i < n; i++) {
        for(int j = 0; j < n; j++){
          printf("%d ", h_c[i * n + j]);
        } printf("\n");
    } printf("\n");

    // Free memory
    free(h_a); free(h_b); free(h_c);
    cudaFree(d_a); cudaFree(d_b); cudaFree(d_c);

    return 0;
}



Overwriting matrix_mul.cu


In [27]:
!nvcc matrix_mul.cu -o matrix_mul
!./matrix_mul

Matrix A:
0 1 2 3 
4 5 6 7 
8 9 10 11 
12 13 14 15 

Matrix B:
0 1 2 3 
4 5 6 7 
8 9 10 11 
12 13 14 15 

Product:
56 62 68 74 
152 174 196 218 
248 286 324 362 
344 398 452 506 

